In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import seaborn as sns
from collections import Counter

Concatenar tablas

In [ ]:
d1 = pd.read_csv('data/SL132001-data (11).csv')
d2 = pd.read_csv('data/SL132001-data (12).csv')

In [ ]:
data = pd.concat([d1.iloc[1:], d2[1:]], axis=0, ignore_index=True)
data

Recortar Datos

In [ ]:
#eliminar columnas
data.drop(columns=['Date', 'Time GMT -4', 'Latitude', 'Longitude', 'Unnamed: 16'], inplace=True)

In [ ]:
#convertir timestamp
data['Timestamp'] = pd.to_datetime(data.Timestamp, unit='ms')
data.set_index('Timestamp', inplace=True)
data.index = data.index.tz_localize('UTC').tz_convert('US/Eastern')
data.head()

In [ ]:
intervals = []
for i in range(1, len(data.index)):
    intervals.append((data.index[i] - data.index[i-1]).seconds)

In [ ]:
cnt = Counter(intervals)
df_cnt = pd.DataFrame(sorted(cnt.items(), key=lambda x: x[1], reverse=True))
df_cnt.columns = ['Interval (s)', 'Counts']
df_cnt.head(5)

In [ ]:
print(min(intervals), max(intervals))

# exploración

series de tiempo (no normalizados y normalizados)

In [ ]:
# grafica de cada caracteristica vs tiempo
fig, ax = plt.subplots(figsize=(16, 6))
for col in data.columns:
    data[col] = data[col].astype('float64')    
    data.plot(y=col, use_index=True, ax=ax)
plt.show()

In [ ]:
# scaler = StandardScaler()
# scaler = RobustScaler()
scaler = MinMaxScaler()
data_norm = data.copy()
data_norm[:] = scaler.fit_transform(data_norm)

In [ ]:
# grafica de cada caracteristica (normalizada) vs tiempo
fig, ax = plt.subplots(figsize=(16, 6))
for col in data_norm.columns:
    data_norm[col] = data_norm[col].astype('float64')    
    data_norm.plot(y=col, use_index=True, ax=ax)
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize=(16, 8))
cols = [9, 10, 0]
colors = ['tab:blue', 'tab:orange', 'tab:green']
for i in range(3):
    ax = axs[i]
    col = data_norm.columns[cols[i]]
    ax.plot(data_norm.index.strftime("%m/%d/%Y, %H:%M:%S"), data_norm[col], color=colors[i], label=col)
    ax.set_ylabel('Normalized value')
    ax.set_xlim([data_norm.index[0], data_norm.index[-1]])
    ax.legend()
    ax.axvspan('2020-05-26 09:00:00-04:00', '2020-05-28 9:00:00-04:00', alpha=0.05, color='red')
    ax.axvspan('2020-05-29 18:00:00-04:00', '2020-06-01 8:00:00-04:00', alpha=0.05, color='red')
axs[0].set_title('Measurements of Ozone, Temperature (External) and Humidity (External)')
plt.xlabel('Timestamp')
plt.show()

